In [1]:
import pandas as pd
import numpy as np
import gzip as gzip
import matplotlib.pyplot as plt
from datetime import datetime, date, timedelta
import gc

In [2]:
#pip install pyarrow

In [3]:
gc.collect()

50

## Read in CoinMetrics Data

In [58]:
#Data downloaded from here https://coinmetrics.io/community-network-data/
infile='C:/Users/19084/My Backup Files/Digital Assets/eth.csv' #'F:/Proj/Digital/DigiCross/Data/Factors/coinmetrics_eth.csv.gz'
coinmetrics_df=pd.read_csv(infile)
coinmetrics_df['pre_SplyCur']=coinmetrics_df['SplyCur'].shift(1)
coinmetrics_df['BlkCntMA']=coinmetrics_df['BlkCnt'].rolling(window=5).mean()
coinmetrics_basic_df=coinmetrics_df[['time','AdrActCnt','BlkCnt','BlkCntMA','BlkSizeMeanByte','pre_SplyCur','SplyCur', \
                                     'DiffMean','IssContNtv','TxCnt','TxTfrCnt','HashRate']].copy()
coinmetrics_sample_df2=coinmetrics_basic_df.query('time>="2022-01-01" and time <="2022-06-30"')
# This avoids a lot of setting with copy warnings
coinmetrics_sample_df3=coinmetrics_sample_df2.reset_index()
coinmetrics_sample_df4=coinmetrics_sample_df3.drop(columns=['index'])
coinmetrics_sample_df=coinmetrics_sample_df4.copy()

## Set Time Interval

In [83]:
mins =  30 #set minutes here

min_diff = pd.Timedelta(mins,unit='m')
intervals = int(1440/mins)

In [84]:
## Things to estimate
minmetrics_sample_df = pd.DataFrame(index=range(0,intervals*len(coinmetrics_sample_df))) 
minmetrics_sample_df['Time']=0
minmetrics_sample_df['AdrActCnt_hat']=0
minmetrics_sample_df['BlkCnt_hat']=0
minmetrics_sample_df['BlkSizeMeanByte_hat']=0
minmetrics_sample_df['DiffMean_hat']=0
minmetrics_sample_df['IssContNtv_hat']=0
minmetrics_sample_df['SplyCur_hat']=0
minmetrics_sample_df['TxCnt_hat']=0
minmetrics_sample_df['TxTfrCnt_hat']=0
minmetrics_sample_df['HashRate_hat']=0

In [85]:
#infile='C:/Users/19084/My Backup Files/Digital Assets/ETH/blocks2021/202110/blocks_20211030.csv.gz'
#temp=pd.read_csv(infile)
#temp['times']=temp['timestamp'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S%z").replace(tzinfo=None))


In [86]:
#temp.head()

In [87]:
#temp['b_diff']=(temp['times']-temp['times'].shift(1))/1000

In [88]:
#x=temp['b_diff'].sum()/(len(temp)-1)
#timedelta.total_seconds(x)

In [89]:
#x=temp['block_timestamp'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S%z").replace(tzinfo=None))
#x[0]

In [90]:
proj_dir='C:/Users/19084/My Backup Files/Digital Assets/ETH/'#'F:/Proj/Digital/Network/Data/Ether/'
date_list=pd.date_range("2022-01-01","2022-06-30",freq='D')
for j in range(0,len(date_list)):
    sdate=str(date_list[j])[0:10]
    print(sdate)
    sint=sdate.replace('-','')
    syear=str(sint)[0:4]
    smonth=str(sint)[4:6]
    sday=str(sint)[6:8]
    infile=proj_dir + 'transactions' + syear + '/' + syear + smonth + '/transactions_' + sint + '.parquet'#'Transactions/Dataframe/' + syear + '/' + smonth + '/transactions_' + sint + '.csv.gz'
    trans_df=pd.read_parquet(infile)
    infile=proj_dir + 'token_transactions' + syear + '/' + syear + smonth + '/trans_tokens_' + sint + '.parquet'#'Token_Transfers/Dataframe/' + syear + '/' + smonth + '/token_transfers_' + sint + '.csv.gz'
    token_df=pd.read_parquet(infile)
    infile=proj_dir + 'blocks' + syear + '/' + syear + smonth + '/blocks_' + sint + '.csv.gz'#'Blocks/Dataframe/' + syear + '/' + smonth + '/blocks_' + sint + '.csv.gz'
    blocks_df=pd.read_csv(infile)
    #blocks_df['timediff']=(blocks_df['block_timestamp']-blocks_df['block_timestamp'].shift(1))/1000
    infile=proj_dir + 'traces' + syear + '/' + syear + smonth + '/traces_' + sint + '.parquet'#'Traces/Dataframe/' + syear + '/' + smonth +'/' + 'traces_' + sint + '.csv.gz'
    traces_df=pd.read_parquet(infile)
    traces_df['values']=traces_df['value'].apply(lambda x: int(x)/1e18)

    
    pre_sup_cur = coinmetrics_sample_df['pre_SplyCur'].iloc[j]
    start_time=date_list[j]
    #minmetrics_sample_df['Time'].iloc[j]=time
    
    trans_df['times']=trans_df['block_timestamp'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").replace(tzinfo=None))
            #(x, "%Y-%m-%d %H:%M:%S%z").replace(tzinfo=None)) 01,   #(x, "%Y-%m-%d %H:%M:%S %Z")) 04, one with no z 05
    token_df['times']=token_df['block_timestamp'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").replace(tzinfo=None))
                #(x, "%Y-%m-%d %H:%M:%S%z").replace(tzinfo=None))   #(x, "%Y-%m-%d %H:%M:%S %Z")), one with no z

    blocks_df['times']=blocks_df['block_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").replace(tzinfo=None))
    blocks_df['timediff']=(blocks_df['times']-blocks_df['times'].shift(1))
    traces_df['times']=traces_df['block_timestamp'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S%z").replace(tzinfo=None))

    for i in range (0,intervals):
        #end_time = start_time+min_diff
        start_time_temp = start_time+i*min_diff
        end_time = start_time+(i+1)*min_diff

        temp_trans_df=trans_df.loc[(trans_df['times'] > start_time_temp) & (trans_df['times']  <= end_time)]
        temp_token_df=token_df.loc[(token_df['times'] > start_time_temp) & (token_df['times']  <= end_time)]
        temp_blocks_df=blocks_df.loc[(blocks_df['times'] > start_time_temp) & (blocks_df['times']  <= end_time)]
        temp_traces_df=traces_df.loc[(traces_df['times'] > start_time_temp) & (traces_df['times']  <= end_time)]
        
        #Time Count
        minmetrics_sample_df['Time'].iloc[j*intervals+i]=start_time_temp
        
        #Average Block Time
        Av_Block_Time=timedelta.total_seconds(temp_blocks_df['timediff'].sum())/(len(temp_blocks_df)-1)
        
        rewards_df=temp_traces_df.query('trace_type=="reward"')
        transfer_df=temp_traces_df.query('values>0 and trace_type=="call" and from_address!=to_address')
        suicide_df=temp_traces_df.query('trace_type=="suicide"')
        
        # Active Address Count
        ether_from_address_list=temp_trans_df['from_address'].to_list()
        ether_to_address_list=temp_trans_df['to_address'].to_list()
        token_from_address_list=temp_token_df['token_from_address'].to_list()
        token_to_address_list=temp_token_df['token_to_address'].to_list()
        address_list=ether_from_address_list+ether_to_address_list+token_from_address_list+token_to_address_list
        address_ulist=np.unique(list(filter(lambda item: item is not None, address_list)))
        minmetrics_sample_df['AdrActCnt_hat'].iloc[j*intervals+i]=len(address_ulist)
        
        # Block Count
        minmetrics_sample_df['BlkCnt_hat'].iloc[j*intervals+i]=temp_blocks_df['block_number'].count()
        # Block Size
        minmetrics_sample_df['BlkSizeMeanByte_hat'].iloc[j*intervals+i] = temp_blocks_df['size'].sum()/len(temp_blocks_df['size'])
    
        # Mean Difficulty
        minmetrics_sample_df['DiffMean_hat'].iloc[j*intervals+i] = temp_blocks_df['difficulty'].mean()
    
        # New Issuance from Miner Rewards
        minmetrics_sample_df['IssContNtv_hat'].iloc[j*intervals+i]=rewards_df['values'].sum()
        minmetrics_sample_df['SplyCur_hat'].iloc[j*intervals+i]=pre_sup_cur + minmetrics_sample_df['IssContNtv_hat'].iloc[j*intervals+i]
        pre_sup_cur = minmetrics_sample_df['SplyCur_hat'].iloc[j*intervals+i]
    
        # Transaction count
        minmetrics_sample_df['TxCnt_hat'].iloc[j*intervals+i]=temp_blocks_df['transaction_count'].sum()
    
        # Transaction transfer count
        minmetrics_sample_df['TxTfrCnt_hat'].iloc[j*intervals+i]=len(transfer_df)-len(suicide_df)
    
        # HashRate
        minmetrics_sample_df['HashRate_hat'].iloc[j*intervals+i]= \
        (minmetrics_sample_df['DiffMean_hat'].iloc[j*intervals+i] /Av_Block_Time)/(10**12)
        
        #start_time=end_time

2022-01-01


C:\Users\19084\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


2022-01-02
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
2022-01-08
2022-01-09
2022-01-10
2022-01-11
2022-01-12
2022-01-13
2022-01-14
2022-01-15
2022-01-16
2022-01-17
2022-01-18
2022-01-19
2022-01-20
2022-01-21
2022-01-22
2022-01-23
2022-01-24
2022-01-25
2022-01-26
2022-01-27
2022-01-28
2022-01-29
2022-01-30
2022-01-31
2022-02-01
2022-02-02
2022-02-03
2022-02-04
2022-02-05
2022-02-06
2022-02-07
2022-02-08
2022-02-09
2022-02-10
2022-02-11
2022-02-12
2022-02-13
2022-02-14
2022-02-15
2022-02-16
2022-02-17
2022-02-18
2022-02-19
2022-02-20
2022-02-21
2022-02-22
2022-02-23
2022-02-24
2022-02-25
2022-02-26
2022-02-27
2022-02-28
2022-03-01
2022-03-02
2022-03-03
2022-03-04
2022-03-05
2022-03-06
2022-03-07
2022-03-08
2022-03-09
2022-03-10
2022-03-11
2022-03-12
2022-03-13
2022-03-14
2022-03-15
2022-03-16
2022-03-17
2022-03-18
2022-03-19
2022-03-20
2022-03-21
2022-03-22
2022-03-23
2022-03-24
2022-03-25
2022-03-26
2022-03-27
2022-03-28
2022-03-29
2022-03-30
2022-03-31
2022-04-01
2022-04-02

In [91]:
#temp_blocks_df.head()

In [92]:
minmetrics_sample_df = minmetrics_sample_df[minmetrics_sample_df['Time'] != 0]

In [93]:
minmetrics_sample_df.to_csv('C:/Users/19084/My Backup Files/Digital Assets/ETH/ETH_2022_30min0101-0631.csv')

In [94]:
minmetrics_sample_df.tail()

,Time,AdrActCnt_hat,BlkCnt_hat,BlkSizeMeanByte_hat,DiffMean_hat,IssContNtv_hat,SplyCur_hat,TxCnt_hat,TxTfrCnt_hat,HashRate_hat
8683,2022-06-30 21:30:00,21514,139,89302.755396,11933363558942452,293.5625,1.188591e+08,26375,20997,908.832324
8684,2022-06-30 22:00:00,20738,138,89789.181159,12001380493546840,292.0625,1.188594e+08,25129,25163,919.568863
8685,2022-06-30 22:30:00,19704,100,89600.920000,11882174431939684,212.6875,1.188596e+08,22737,23001,656.437092
8686,2022-06-30 23:00:00,17806,121,93531.239669,11702786482629868,250.8125,1.188599e+08,23540,29140,779.752570
8687,2022-06-30 23:30:00,16754,118,107840.296610,11504655432186798,256.7500,1.188601e+08,20741,19893,753.242689


In [29]:
coinmetrics_sample_df.head()

,time,AdrActCnt,BlkCnt,BlkCntMA,BlkSizeMeanByte,pre_SplyCur,SplyCur,DiffMean,IssContNtv,TxCnt,TxTfrCnt,HashRate
0,2021-07-01,616892.0,6350.0,6425.0,54672.258583,1.165083e+08,1.165215e+08,6.334799e+15,13238.3750,1211710.0,921446.0,465.612884
1,2021-07-02,631727.0,6437.0,6422.8,52887.842784,1.165215e+08,1.165349e+08,6.350688e+15,13422.7500,1181574.0,859860.0,473.346861
2,2021-07-03,656263.0,6417.0,6413.0,51333.376032,1.165349e+08,1.165483e+08,6.373935e+15,13335.3125,1122543.0,819978.0,473.455196
3,2021-07-04,655655.0,6458.0,6423.4,52285.702849,1.165483e+08,1.165617e+08,6.385745e+15,13452.5625,1105343.0,821989.0,477.374624
4,2021-07-05,620458.0,6419.0,6416.2,53362.713507,1.165617e+08,1.165751e+08,6.478657e+15,13417.1250,1147895.0,824330.0,481.422948


## Original Code

In [3]:
coinmetrics_sample_df['AdrActCnt_hat']=0
coinmetrics_sample_df['BlkCnt_hat']=0
coinmetrics_sample_df['BlkSizeMeanByte_hat']=0
coinmetrics_sample_df['DiffMean_hat']=0
coinmetrics_sample_df['IssContNtv_hat']=0
coinmetrics_sample_df['SplyCur_hat']=0
coinmetrics_sample_df['TxCnt_hat']=0
coinmetrics_sample_df['TxTfrCnt_hat']=0
coinmetrics_sample_df['HashRate_hat']=0
coinmetrics_sample_df.head()

,date,AdrActCnt,BlkCnt,BlkCntMA,BlkSizeMeanByte,pre_SplyCur,SplyCur,DiffMean,IssContNtv,TxCnt,...,HashRate,AdrActCnt_hat,BlkCnt_hat,BlkSizeMeanByte_hat,DiffMean_hat,IssContNtv_hat,SplyCur_hat,TxCnt_hat,TxTfrCnt_hat,HashRate_hat
0,2021-01-01,511250,6545,6519.6,40957.125898,1.140650e+08,1.140786e+08,3.730319e+15,13534.1875,1136712,...,282.589492,0,0,0,0,0,0,0,0,0


In [4]:
proj_dir='C:/Users/19084/My Backup Files/Digital Assets/ETH/'#'F:/Proj/Digital/Network/Data/Ether/'
date_list=pd.date_range("2021-07-01","2021-07-01",freq='D')
for j in range(0,len(date_list)):
    sdate=str(date_list[j])[0:10]
    print(sdate)
    sint=sdate.replace('-','')
    syear=str(sint)[0:4]
    smonth=str(sint)[4:6]
    sday=str(sint)[6:8]
    infile=proj_dir + 'transactions_' + sint + '.csv.gz'#'Transactions/Dataframe/' + syear + '/' + smonth + '/transactions_' + sint + '.csv.gz'
    trans_df=pd.read_csv(infile)
    infile=proj_dir + 'token_transfers_' + sint + '.csv.gz'#'Token_Transfers/Dataframe/' + syear + '/' + smonth + '/token_transfers_' + sint + '.csv.gz'
    token_df=pd.read_csv(infile)
    infile=proj_dir + 'blocks_' + sint + '.csv.gz'#'Blocks/Dataframe/' + syear + '/' + smonth + '/blocks_' + sint + '.csv.gz'
    blocks_df=pd.read_csv(infile)
    blocks_df['timediff']=(blocks_df['block_timestamp']-blocks_df['block_timestamp'].shift(1))/1000
    Av_Block_Time=blocks_df['timediff'].sum()/(len(blocks_df)-1)
    infile=proj_dir +'traces_' + sint + '.csv.gz'#'Traces/Dataframe/' + syear + '/' + smonth +'/' + 'traces_' + sint + '.csv.gz'
    traces_df=pd.read_csv(infile)
    traces_df['values']=traces_df['value'].apply(lambda x: int(x)/1e18)
    rewards_df=traces_df.query('trace_type=="reward"')
    transfer_df=traces_df.query('values>0 and trace_type=="call" and from_address!=to_address')
    suicide_df=traces_df.query('trace_type=="suicide"')

    # Active Address Count
    ether_from_address_list=trans_df['from_address'].to_list()
    ether_to_address_list=trans_df['to_address'].to_list()
    token_from_address_list=token_df['from_address'].to_list()
    token_to_address_list=token_df['to_address'].to_list()
    address_list=ether_from_address_list+ether_to_address_list+token_from_address_list+token_to_address_list
    address_ulist=np.unique(address_list)
    coinmetrics_sample_df['AdrActCnt_hat'].iloc[j]=len(address_ulist)
    
    # Block Count
    coinmetrics_sample_df['BlkCnt_hat'].iloc[j]=blocks_df['block_number'].count()
    # Block Size
    coinmetrics_sample_df['BlkSizeMeanByte_hat'].iloc[j] = blocks_df['size'].sum()/len(blocks_df['size'])
    
    # Mean Difficulty
    coinmetrics_sample_df['DiffMean_hat'].iloc[j] = blocks_df['difficulty'].mean()
    
    # New Issuance from Miner Rewards
    coinmetrics_sample_df['IssContNtv_hat'].iloc[j]=rewards_df['values'].sum()
    coinmetrics_sample_df['SplyCur_hat'].iloc[j]=coinmetrics_sample_df['pre_SplyCur'].iloc[j] + coinmetrics_sample_df['IssContNtv_hat'].iloc[j]
    
    # Transaction count
    coinmetrics_sample_df['TxCnt_hat'].iloc[j]=blocks_df['transaction_count'].sum()
    
    # Transaction transfer count
    coinmetrics_sample_df['TxTfrCnt_hat'].iloc[j]=len(transfer_df)-len(suicide_df)
    
    # HashRate
    coinmetrics_sample_df['HashRate_hat'].iloc[j]= \
        (coinmetrics_sample_df['DiffMean_hat'].iloc[j] /Av_Block_Time)/(10**12)


2021-01-01


C:\Users\Karen\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
concept_list=['AdrActCnt','BlkCnt','BlkSizeMeanByte','DiffMean','IssContNtv','SplyCur','TxCnt','TxTfrCnt','HashRate']
for j in range(0,len(concept_list)):
    cname1=concept_list[j]
    cname2=concept_list[j]+ '_hat'
    cname3=concept_list[j]+ '_error'
    coinmetrics_sample_df[cname3]=coinmetrics_sample_df[cname1]-coinmetrics_sample_df[cname2]
    mean_error=coinmetrics_sample_df[cname3].mean()
    pct_error=coinmetrics_sample_df[cname3].mean()/coinmetrics_sample_df[cname1].mean()
    print(concept_list[j],"Avg. error = ",mean_error,"Pct. error = ",'{:.2f}'.format(100*pct_error))

AdrActCnt Avg. error =  -9341.0 Pct. error =  -1.83
BlkCnt Avg. error =  0.0 Pct. error =  0.00
BlkSizeMeanByte Avg. error =  7.275957614183426e-12 Pct. error =  0.00
DiffMean Avg. error =  -11.5 Pct. error =  -0.00
IssContNtv Avg. error =  0.0 Pct. error =  0.00
SplyCur Avg. error =  0.0 Pct. error =  0.00
TxCnt Avg. error =  0.0 Pct. error =  0.00
TxTfrCnt Avg. error =  -12508.0 Pct. error =  -1.95
HashRate Avg. error =  -5.0943299356731586e-08 Pct. error =  -0.00
